In [57]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2

try:
    from PIL import Image
except ImportError:
    import Image

In [58]:
IMAGES_DIR_PATH="./random-cars-dataset/images"
LABELS_DIR_PATH="./random-cars-dataset/license-numbers"

In [59]:
import os
images=[f for f in os.listdir(IMAGES_DIR_PATH)]

In [4]:
img_path=f'{IMAGES_DIR_PATH}/'

#predicting labels for all test images
%cd yolov5
!python detect.py --weights ./runs/train/exp/weights/best.pt --source ../{img_path} --save-txt --exist-ok
%cd ..

C:\Users\HP\Desktop\ML\ANPR\yolov5
Namespace(agnostic_nms=False, augment=False, classes=None, conf_thres=0.25, device='', exist_ok=True, img_size=640, iou_thres=0.45, name='exp', nosave=False, project='runs/detect', save_conf=False, save_txt=True, source='.././random-cars-dataset/images/', update=False, view_img=False, weights=['./runs/train/exp/weights/best.pt'])
Fusing layers... 
image 1/84 C:\Users\HP\Desktop\ML\ANPR\yolov5\..\random-cars-dataset\images\Cars0.png: 352x640 1 license_plate, Done. (0.429s)
image 2/84 C:\Users\HP\Desktop\ML\ANPR\yolov5\..\random-cars-dataset\images\Cars1.png: 416x640 1 license_plate, Done. (0.271s)
image 3/84 C:\Users\HP\Desktop\ML\ANPR\yolov5\..\random-cars-dataset\images\Cars102.png: 512x640 1 license_plate, Done. (0.295s)
image 4/84 C:\Users\HP\Desktop\ML\ANPR\yolov5\..\random-cars-dataset\images\Cars107.png: 384x640 1 license_plate, Done. (0.205s)
image 5/84 C:\Users\HP\Desktop\ML\ANPR\yolov5\..\random-cars-dataset\images\Cars108.png: 512x640 1 lice

YOLOv5  5352f33 torch 1.8.1+cpu CPU

Model Summary: 224 layers, 7053910 parameters, 0 gradients, 16.3 GFLOPS


image 80/84 C:\Users\HP\Desktop\ML\ANPR\yolov5\..\random-cars-dataset\images\Cars60.png: 480x640 1 license_plate, Done. (0.277s)
image 81/84 C:\Users\HP\Desktop\ML\ANPR\yolov5\..\random-cars-dataset\images\Cars65.png: 544x640 1 license_plate, Done. (0.372s)
image 82/84 C:\Users\HP\Desktop\ML\ANPR\yolov5\..\random-cars-dataset\images\Cars67.png: 320x640 1 license_plate, Done. (0.210s)
image 83/84 C:\Users\HP\Desktop\ML\ANPR\yolov5\..\random-cars-dataset\images\Cars77.png: 448x640 1 license_plate, Done. (0.275s)
image 84/84 C:\Users\HP\Desktop\ML\ANPR\yolov5\..\random-cars-dataset\images\Cars86.png: 384x640 1 license_plate, Done. (0.227s)
Results saved to runs\detect\exp
85 labels saved to runs\detect\exp\labels
Done. (33.635s)


In [60]:
def get_yolo_label(f_name):
    #opening label file
    label_file=open(f'./yolov5/runs/detect/exp/labels/{f_name}.txt')

    #reading line
    s=label_file.readline().replace('\n','').split(' ')

    #******extracting bounding box********#
    s=[ float(i) for i in s]    #[conf center_x center_y width height] 

    bottom_x=((2*s[1]+s[3])/2)
    top_x=bottom_x-s[3]

    bottom_y=((2*s[2]+s[4])/2)
    top_y=bottom_y-s[4]

    top_x=int(top_x*width)
    bottom_x=int(bottom_x*width)

    top_y=int(top_y*height)
    bottom_y=int(bottom_y*height)
    #*************************************#

    return [top_x,top_y,bottom_x,bottom_y]
   

In [120]:
def pre_process(plate):
    # resize image to three times as large as original for better readability
    plate=cv2.resize(plate,(1000,600),interpolation=cv2.INTER_CUBIC)
    
    #blur
    #plate=cv2.GaussianBlur(plate, (5,5), 0)
    plate=cv2.bilateralFilter(plate,9,2,100)
    
    #converting to gray scale
    plate=cv2.cvtColor(plate, cv2.COLOR_BGR2GRAY)
    
    #increasing contrast
    plate=cv2.multiply(plate,1.5)

    #binarizing
    #plate=cv2.threshold(plate, 0, 255, cv2.THRESH_OTSU | cv2.THRESH_BINARY_INV)[1]
    plate=cv2.adaptiveThreshold(plate,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C,cv2.THRESH_BINARY,199,5)
    
    return plate

In [121]:
def segment_chars(plate):
    contours,_=cv2.findContours(plate, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    contours = sorted(contours, key=lambda ctr: cv2.boundingRect(ctr)[0])
    chars=[]
    widths=[]
    heights=[]

    for cont in contours:
        x,y,w,h=cv2.boundingRect(cont)

        #w: width of character
        #h height of character

        #shape of plate
        height_of_plate,width_of_plate=plate.shape

        #.......removing noise........#

        #if char is too wide then skip
        if width_of_plate<2*w: continue        

        #if char is too short in height then skip
        if height_of_plate>8*h: continue

        #if char h/w ratio is less than 1 or more than 10
        ratio=h/float(w)
        if ratio<0.5 or ratio>10: continue

        #if area is less than 100 pixels
        if h*w<70: continue

        #.............................#

        #checking if contour inside already added contour
        f=0
        for c in chars:
            x1,y1,w1,h1=cv2.boundingRect(c)
            if x>x1 and x+w<x1+w1 and y>y1 and y+h<y1+h1:
                #contour not useful, mark flag to 1
                f=1
                break

        if f==1: continue
        #..................................................
            
        #check if its in corner
        if x==0 or y==0 or x+w==width_of_plate or y+h==height_of_plate:
            continue    
        #.....................


        #appending valid contour
        chars.append(cont)
        widths.append(w)
        heights.append(h)
        
    #comparing height and width of charactes with median height and width
    from statistics import median
    mw=median(widths)
    mh=median(heights)

    def is_hw_ok(cont):
        x,y,w,h=cv2.boundingRect(cont)

        if h/mh<0.55 or h/mh>2 or w/mw<0.33 or w/mw>3: return False

        return True

    chars=[char for char in chars if is_hw_ok(char)]

    #......................................................................
    
    return chars

In [122]:
#utility function to convert number(0-35) to label(0-9, A-Z)
def to_label(n):
    if n<=9:
        return str(n)
    
    return chr(n-10+ord('A'))


from keras.models import load_model

def predict_label(chars,char_recognizer):
    label=""

    #iterating over contours
    for char in chars:
        x,y,w,h=cv2.boundingRect(char) #w: width of character, h: height of character

        roi=plate[y:y+h,x:x+w]

#         point=plate[char[0][0][1]+1][char[0][0][0]]

#         #if char is white
#         if point==255:
#             roi=cv2.bitwise_not(roi)

        #adding padding
        #roi=cv2.copyMakeBorder(roi,8,8,8,8,cv2.BORDER_CONSTANT,value=[255,255,255])
        
        roi=cv2.normalize(roi,None,0,1,cv2.NORM_MINMAX)

        #resizing image to model input format (28x28)
        roi=cv2.resize(roi,(28,28))

        #converting roi into required format
        roi=roi.reshape(28,28,1)

        #predicting label using model
        pred=char_recognizer.predict(np.array([roi]))

        #finding label mith max probability
        pred=np.argmax(pred)

        label+=to_label(pred)
        
    return label

In [123]:
from difflib import SequenceMatcher
import jellyfish

def cal_score(label_pred,label_true):
    
    #find similarity between two strings
    sim=SequenceMatcher(None,label_pred,label_true).ratio()   #o to 1
    
    #edit distance
    edit_dist=jellyfish.levenshtein_distance(label_pred,label_true)
    
    #segmentation error
    seg_err=len(label_true)-len(label_pred)
    
    return [sim,edit_dist,seg_err]

def final_score(scores):
    #[sim,edit_text]
    avg_sim=0
    avg_ed=0
    avg_seg_err=0
    for s in scores:
        avg_sim+=s[0]
        avg_ed+=s[1]
        avg_seg_err+=abs(s[2])
        
    avg_sim/=len(scores)
    avg_ed/=len(scores)
    avg_seg_err/=len(scores)
        
    print(f"\nAverage Segmentation Error: {avg_seg_err}\nAverage similarity: {avg_sim}\nAverage Edit Distance: {avg_ed}")
        

In [124]:
char_recognizer=load_model('lenet_50.h5')

scores=[]
bad=[]
for i,image in enumerate(images):
    #image path
    img_path=f'{IMAGES_DIR_PATH}/{image}'
    
    #image name
    f_name=image.split('.')[0]

    #getting image size
    im = Image.open(img_path)
    width,height= im.size
    
    #reading image
    img=cv2.imread(img_path)
    
    #detecting license plate using yolov5 model
    [top_x,top_y,bottom_x,bottom_y]=get_yolo_label(f_name)
    
    #cropping the license plate from the original image
    plate=img[top_y:bottom_y,top_x:bottom_x]
    
    #apply pre-processing on plate
    plate=pre_process(plate)
    
    #character segmentation
    chars=segment_chars(plate)
    
    #predict label
    label_pred=predict_label(chars,char_recognizer)
    
    #read true label
    f1=open(f'./random-cars-dataset/license_numbers/{f_name}.txt')
    label_true=f1.readline()
    
    #cal score
    score=cal_score(label_pred,label_true)
    
    scores.append(score)
    
    print(f"#{i} {f_name}\tSE:{score[2]}\tSim: {round(score[0],2)}\tED: {score[1]}\ttrue: {label_true}\tpred: {label_pred}")
    
    if score[0]<0.5:
        bad.append(f_name)
    
#display average scores
final_score(scores)

#0 Cars0	SE:0	Sim: 0.7	ED: 3	true: KL01CA2555	pred: KEZ1VA2555
#1 Cars1	SE:-2	Sim: 0.88	ED: 2	true: PGMN112	pred: KPG0MN112
#2 Cars102	SE:0	Sim: 1.0	ED: 0	true: 6861136	pred: 6861136
#3 Cars107	SE:-1	Sim: 0.47	ED: 5	true: MH20BQ20	pred: MHPDFBQPD
#4 Cars108	SE:0	Sim: 0.9	ED: 1	true: MH01AV8866	pred: MH01AY8866
#5 Cars109	SE:0	Sim: 0.57	ED: 3	true: CZ17KOD	pred: UZ17KU1
#6 Cars11	SE:1	Sim: 0.77	ED: 2	true: WOR516K	pred: W0R56K
#7 Cars111	SE:0	Sim: 1.0	ED: 0	true: MH20EE7598	pred: MH20EE7598
#8 Cars113	SE:0	Sim: 0.86	ED: 1	true: MPEACHW	pred: MPEAUHW
#9 Cars116	SE:0	Sim: 0.67	ED: 2	true: MK3532	pred: YK353E
#10 Cars118	SE:0	Sim: 0.86	ED: 1	true: JA62UAR	pred: JA52UAR
#11 Cars12	SE:0	Sim: 0.9	ED: 1	true: MH12BG7237	pred: HH12BG7237
#12 Cars120	SE:-1	Sim: 0.8	ED: 2	true: B2228HM	pred: KB222BHM
#13 Cars125	SE:2	Sim: 0.67	ED: 3	true: RP66887	pred: X6688
#14 Cars126	SE:0	Sim: 0.7	ED: 3	true: KA05MG1909	pred: KAU5M61904
#15 Cars128	SE:0	Sim: 0.86	ED: 1	true: 0X65AWD	pred: BX65AWD
#16 Cars129	S

In [126]:
bad

['Cars107', 'Cars309', 'Cars430']